# Recommendation Pipeline 상세 설명
이 파일은 사용자 선호도, 예산, 시청 습관을 기반으로 OTT 플랫폼과 콘텐츠를 추천하는 시스템을 구현합니다. 이 시스템은 다음과 같은 파이프라인을 따릅니다.

## 1. 데이터 로드 및 전처리
- **데이터 로드**: 남성, 여성 데이터셋과 OTT 가격 데이터를 로드합니다.
- **플랫폼 열 확장**: 여러 플랫폼이 포함된 열을 개별 행으로 확장하여 각 플랫폼에 대한 콘텐츠를 분리합니다.
- **플랫폼-콘텐츠 매핑 생성**: 각 플랫폼에 대해 사용 가능한 콘텐츠를 매핑합니다.

## 2. 사용자 프로필 생성
- **입력 데이터**: 사용자의 성별, 연령대, 선호 장르, 세부 장르, 주간 시청 시간, 월간 예산을 입력받습니다.
- **프로필 생성**: 입력 데이터를 기반으로 사용자 프로필을 생성합니다.

## 3. 콘텐츠 추천
- **TF-IDF 기반 유사도 계산**:
  - 콘텐츠의 세부 장르 데이터를 텍스트로 결합하여 TF-IDF 행렬을 생성합니다.
  - 사용자의 선호 장르를 기반으로 사용자 벡터를 생성합니다.
  - 코사인 유사도를 계산하여 사용자 선호도와 유사한 콘텐츠를 찾습니다.
- **필터링**:
  - 성별, 연령대, 선호 장르에 따라 데이터셋을 필터링합니다.
  - 필터링 후 데이터셋이 너무 작으면 원본 데이터셋을 사용합니다.
- **결과 정렬**: 유사도와 점수를 기준으로 콘텐츠를 정렬하고 상위 N개의 콘텐츠를 반환합니다.

## 4. 플랫폼 추천
- **플랫폼별 콘텐츠 수 계산**: 추천된 콘텐츠를 제공하는 플랫폼별로 콘텐츠 수를 계산합니다.
- **가격 정보 매핑**: 각 플랫폼의 요금제를 매핑하여 가격 정보를 추가합니다.
- **최적화**:
  - 예산 내에서 가장 많은 콘텐츠를 제공하는 플랫폼을 선택합니다.
  - 상위 2개의 플랫폼을 선택하며, 예산 초과 시 가장 저렴한 옵션을 선택합니다.

## 5. 플랫폼별 콘텐츠 필터링
- **추천 플랫폼 필터링**: 추천된 플랫폼에서 제공하는 콘텐츠만 필터링합니다.
- **부족한 콘텐츠 보완**: 추천 플랫폼에서 충분한 콘텐츠가 없는 경우, 원본 추천 목록에서 추가합니다.

## 6. 구독 전략 생성
- **플랫폼별 콘텐츠 수 계산**: 각 플랫폼에서 제공하는 콘텐츠 수를 계산합니다.
- **구독 전략 생성**:
  - 주요 플랫폼과 보조 플랫폼을 선택합니다.
  - 예산 내에서 두 플랫폼을 동시에 구독하거나, 하나의 플랫폼만 구독하는 전략을 생성합니다.

## 7. 최종 결과 반환
- **콘텐츠 추천**: 사용자 선호도와 유사한 콘텐츠 목록을 반환합니다.
- **플랫폼 추천**: 예산 내에서 최적의 플랫폼 조합을 반환합니다.
- **구독 전략**: 추천된 플랫폼과 콘텐츠를 기반으로 구독 전략을 제공합니다.

# Recommendation Logic
이 섹션에서는 사용자 선호도, 예산 및 시청 습관을 기반으로 OTT 플랫폼과 콘텐츠를 추천하는 로직을 구현합니다.

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
import logging


In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
# 전역 변수 초기화
male_data = None
female_data = None
ott_prices = None
platform_content_map = {}

In [4]:
def expand_platform_column(df):
    """여러 플랫폼이 포함된 플랫폼 열을 개별 행으로 확장"""
    df_expanded = df.copy()
    # 플랫폼 리스트를 정리하여 새 열 생성
    df_expanded['platforms'] = df_expanded['platform'].fillna('').str.split(',').apply(
        lambda x: [platform.strip() for platform in x] if isinstance(x, list) else []
    )
    return df_expanded

In [21]:
def load_data():
    """데이터셋 로드 및 전처리"""
    global male_data, female_data, ott_prices
    try:
        # 현재 작업 디렉토리 가져오기
        current_dir = os.getcwd()
        data_dir = os.path.join(current_dir, 'data')

        # 남성, 여성 데이터 및 OTT 가격 데이터 로드
        male_file = os.path.join(data_dir, 'male_train_250514.csv')
        female_file = os.path.join(data_dir, 'female_train_250514.csv')
        ott_price_file = os.path.join(data_dir, 'ott_price.csv')

        male_data = pd.read_csv(male_file)
        female_data = pd.read_csv(female_file)
        ott_prices = pd.read_csv(ott_price_file)

        logger.info(f"남성 데이터 {len(male_data)}개, 여성 데이터 {len(female_data)}개, OTT 가격 데이터 {len(ott_prices)}개 로드 완료")

        # 플랫폼 데이터를 정리하여 개별 플랫폼 추출
        male_data = expand_platform_column(male_data)
        female_data = expand_platform_column(female_data)

    except Exception as e:
        logger.error(f"데이터 로드 중 오류 발생: {str(e)}")
        raise

In [6]:
def prepare_platform_content_map():
    """플랫폼별 콘텐츠 매핑 생성"""
    global platform_content_map
    # 남성 및 여성 데이터셋 결합
    all_data = pd.concat([male_data, female_data]).drop_duplicates(subset=['title', 'year_x'])

    # 각 플랫폼에 대해 사용 가능한 제목 수집
    for _, row in all_data.iterrows():
        for platform in row['platforms']:
            if platform not in platform_content_map:
                platform_content_map[platform] = []

            platform_content_map[platform].append({
                'title': row['title'],
                'year': row['year_x'],
                'genre': row['genre_x'],
                'genre_detail': row['genre_detail'],
                'score': row['score'],
                'platform': platform
            })

    logger.info(f"{len(platform_content_map)}개의 플랫폼에 대한 콘텐츠 매핑 생성 완료")

In [7]:
def create_user_profile(gender, age_group, preferred_genres, detailed_genres, weekly_hours, monthly_budget):
    """사용자 선호도를 기반으로 사용자 프로필 생성"""
    user_profile = {
        'gender': gender,
        'age_group': age_group,
        'preferred_genres': preferred_genres,
        'detailed_genres': detailed_genres,
        'weekly_hours': weekly_hours,
        'monthly_budget': monthly_budget
    }
    logger.info(f"사용자 프로필 생성: {user_profile}")
    return user_profile

In [8]:
def find_similar_content(user_profile, top_n=6):
    """사용자 선호도와 유사한 콘텐츠 찾기"""
    global male_data, female_data
    # 성별에 따라 데이터셋 선택
    if user_profile['gender'] == '남성':
        dataset = male_data
    else:
        dataset = female_data

    # 연령대에 따라 필터링 (지정된 경우)
    if user_profile['age_group'] != '전체':
        dataset = dataset[dataset['age_group'] == user_profile['age_group']]

    # 주요 장르에 따라 필터링 (지정된 경우)
    if user_profile['preferred_genres'] != ['전체']:
        dataset = dataset[dataset['genre_x'].isin(user_profile['preferred_genres'])]

    # TF-IDF를 위한 장르 텍스트 결합
    dataset['content_text'] = dataset['genre_detail'].fillna('') 

    # 필터링 후 데이터셋이 너무 작으면 원본 데이터셋 사용
    if len(dataset) < top_n:
        logger.warning("필터링 후 데이터셋이 너무 작아 원본 데이터셋 사용")
        if user_profile['gender'] == '남성':
            dataset = male_data
        else:
            dataset = female_data
        dataset['content_text'] = dataset['genre_detail'].fillna('')

    # 세부 장르에 대한 TF-IDF 행렬 생성
    tfidf = TfidfVectorizer(stop_words='english')

    # 데이터셋이 비어 있는 경우 처리
    if len(dataset) == 0:
        logger.warning("사용자 프로필과 일치하는 콘텐츠 없음")
        return []

    tfidf_matrix = tfidf.fit_transform(dataset['content_text'])

    # 사용자 선호도 문서 생성
    user_doc = ', '.join(user_profile['detailed_genres'])
    user_vector = tfidf.transform([user_doc])

    # 유사도 계산
    cosine_sim = cosine_similarity(user_vector, tfidf_matrix).flatten()

    # 상위 유사 콘텐츠의 인덱스 가져오기
    sim_scores_indices = list(enumerate(cosine_sim))
    sim_scores_indices = sorted(sim_scores_indices, key=lambda x: x[1], reverse=True)

    # 상위 N개의 인덱스 가져오기
    top_indices = [i[0] for i in sim_scores_indices[:top_n*2]]  # 다양성을 위해 필요한 것보다 더 많이 가져옴

    # 상위 콘텐츠 가져오기
    top_content = dataset.iloc[top_indices].copy()

    # 보조 기준으로 점수에 따라 정렬
    top_content = top_content.sort_values(by=['score'], ascending=False)

    # 상위 N개 항목만 반환
    top_content = top_content.head(top_n)

    # 딕셔너리 목록으로 변환
    results = []
    for _, row in top_content.iterrows():
        results.append({
            'title': row['title'],
            'year': row['year_x'],
            'genre': row['genre_x'],
            'genre_detail': row['genre_detail'],
            'score': row['score'],
            'platforms': row['platforms']
        })

    logger.info(f"사용자 선호도와 일치하는 콘텐츠 {len(results)}개 발견")
    return results

In [9]:
def optimize_platform_selection(platform_pricing, sorted_platforms, monthly_budget):
    """예산 및 콘텐츠 가용성에 따른 플랫폼 선택 최적화"""
    # 콘텐츠가 가장 풍부한 플랫폼부터 시작
    recommended = []
    total_cost = 0
    monthly_budget = int(monthly_budget)

    # 먼저 상위 2개 플랫폼의 가장 저렴한 요금제 추가 시도
    top_platforms = [p[0] for p in sorted_platforms[:4]]  # 상위 4개 플랫폼 고려

    for platform in top_platforms:
        # 해당 플랫폼의 가장 저렴한 요금제 찾기
        platform_plans = [p for p in platform_pricing if p['platform'].strip() == platform.strip()]
        if platform_plans:
            cheapest_plan = min(platform_plans, key=lambda x: x['price'])

            # 이 플랫폼을 추가하면 예산을 초과하는지 확인
            if total_cost + cheapest_plan['price'] <= monthly_budget:
                recommended.append(cheapest_plan)
                total_cost += cheapest_plan['price']

                # 2개 플랫폼 추가 후 중지
                if len(recommended) >= 2:
                    break

    # 아무 플랫폼도 추가하지 못했거나 하나만 추가된 경우, 가장 저렴한 옵션으로 재시도
    if len(recommended) == 0:
        if platform_pricing:
            cheapest_overall = min(platform_pricing, key=lambda x: x['price'])
            if cheapest_overall['price'] <= monthly_budget:
                recommended.append(cheapest_overall)
                total_cost += cheapest_overall['price']

    # 예산 정보 추가
    result = {
        'platforms': recommended,
        'total_cost': total_cost,
        'remaining_budget': monthly_budget - total_cost
    }

    logger.info(f"추천 플랫폼: {recommended}, 총 비용: {total_cost}, 월 예산: {monthly_budget}")
    return result

In [10]:
def recommend_platforms(content_recommendations, monthly_budget):
    """콘텐츠 추천 및 예산에 따른 OTT 플랫폼 추천"""
    # 추천 콘텐츠가 가장 많은 플랫폼 계산
    platform_counts = {}
    for content in content_recommendations:
        for platform in content['platforms']:
            if platform not in platform_counts:
                platform_counts[platform] = 0
            platform_counts[platform] += 1

    # 콘텐츠 수에 따라 플랫폼 정렬
    sorted_platforms = sorted(platform_counts.items(), key=lambda x: x[1], reverse=True)

    # 플랫폼에 대한 가격 정보 가져오기
    platform_pricing = []
    for platform, _ in sorted_platforms:
        # 플랫폼 이름 정리
        clean_platform = platform.strip()

        # 가격 정보 찾기
        platform_prices = ott_prices[ott_prices['서비스명'].str.contains(clean_platform, case=False, na=False)]

        if not platform_prices.empty:
            for _, price_row in platform_prices.iterrows():
                platform_pricing.append({
                    'platform': clean_platform,
                    'plan': price_row['요금제'],
                    'price': price_row['월 구독료(원)']
                })

    # 예산 내에서 최적의 플랫폼 조합 찾기
    recommended_platforms = optimize_platform_selection(platform_pricing, sorted_platforms, monthly_budget)

    return recommended_platforms

In [11]:
def create_subscription_strategy(content, platform_recommendations):
    """추천 콘텐츠 및 플랫폼에 따른 구독 전략 생성"""
    # 플랫폼별 콘텐츠 수 계산
    platform_counts = {}
    for item in content:
        platform = item['platform']
        if platform not in platform_counts:
            platform_counts[platform] = 0
        platform_counts[platform] += 1

    # 콘텐츠 수에 따라 플랫폼 정렬
    sorted_platforms = sorted(platform_counts.items(), key=lambda x: x[1], reverse=True)

    # 구독 정보 가져오기
    subscriptions = []
    for platform, count in sorted_platforms:
        for p in platform_recommendations['platforms']:
            if p['platform'] == platform:
                subscriptions.append({
                    'platform': platform,
                    'plan': p['plan'],
                    'price': p['price'],
                    'content_count': count
                })

    # 전략 생성
    if len(subscriptions) > 1:
        strategy = {
            'primary_platform': subscriptions[0],
            'secondary_platform': subscriptions[1] if len(subscriptions) > 1 else None,
            'strategy': '월 예산 내에서 두 플랫폼을 동시에 구독하여 콘텐츠 시청'
        }
    else:
        strategy = {
            'primary_platform': subscriptions[0] if subscriptions else None,
            'secondary_platform': None,
            'strategy': '하나의 플랫폼을 구독하여 콘텐츠 시청'
        }

    return strategy

In [12]:
def get_platform_specific_content(content_recommendations, recommended_platforms):
    """추천 플랫폼에 특정한 콘텐츠 가져오기"""
    platform_list = [p['platform'] for p in recommended_platforms['platforms']]

    # 추천 플랫폼에 따라 콘텐츠 필터링
    platform_specific_content = []
    for content in content_recommendations:
        for platform in content['platforms']:
            if platform in platform_list:
                # 플랫폼 특정 콘텐츠 추가
                platform_specific_content.append({
                    'title': content['title'],
                    'year': content['year'],
                    'genre': content['genre'],
                    'genre_detail': content['genre_detail'],
                    'score': content['score'],
                    'platform': platform
                })
                # 여러 플랫폼에서 제공되는 경우 한 번만 추가
                break

    # 추천 플랫폼에서 충분한 콘텐츠가 없는 경우,
    # 원본 추천 목록에서 더 추가
    if len(platform_specific_content) < 6:
        for content in content_recommendations:
            # 아직 추가되지 않은 콘텐츠만 추가
            if not any(pc['title'] == content['title'] for pc in platform_specific_content):
                # 사용 가능한 첫 번째 플랫폼 선택
                if content['platforms']:
                    platform_specific_content.append({
                        'title': content['title'],
                        'year': content['year'],
                        'genre': content['genre'],
                        'genre_detail': content['genre_detail'],
                        'score': content['score'],
                        'platform': content['platforms'][0]
                    })

            # 6개 항목이 되면 중지
            if len(platform_specific_content) >= 6:
                break

    # 구독 전략 정보 추가
    subscription_strategy = create_subscription_strategy(platform_specific_content, recommended_platforms)

    result = {
        'content': platform_specific_content,
        'subscription_strategy': subscription_strategy
    }

    return result

In [13]:
def get_comprehensive_recommendations(user_profile):
    """콘텐츠 및 플랫폼을 포함한 종합 추천 받기"""
    # 유사 콘텐츠 찾기
    content_recommendations = find_similar_content(user_profile)

    # 플랫폼 추천
    platform_recommendations = recommend_platforms(content_recommendations, user_profile['monthly_budget'])

    # 플랫폼 특정 콘텐츠 가져오기
    final_recommendations = get_platform_specific_content(content_recommendations, platform_recommendations)

    # 참조용 사용자 프로필 추가
    final_recommendations['user_profile'] = user_profile

    return final_recommendations

In [24]:
# 추천 시스템 테스트 실행
if __name__ == "__main__":
    # 데이터 로드 및 전처리
    load_data()
    prepare_platform_content_map()

    # 사용자 프로필 생성
    user_profile = create_user_profile(
        gender="남성",
        age_group="20대",
        preferred_genres=["액션", "코미디"],
        detailed_genres=["액션", "코미디", "모험"],
        weekly_hours=10,
        monthly_budget=15000
    )

    # 추천 실행
    recommendations = get_comprehensive_recommendations(user_profile)

    # 결과 출력
    print("추천 콘텐츠:")
    for content in recommendations['content']:
        print(f"- 제목: {content['title']}, 장르: {content['genre']}, 플랫폼: {content['platform']}")

    print("\n추천 플랫폼:")
    if 'platforms' in recommendations['subscription_strategy']:
        for platform in recommendations['subscription_strategy']['platforms']:
            print(f"- 플랫폼: {platform['platform']}, 요금제: {platform['plan']}, 가격: {platform['price']}원")
    else:
        print("추천 플랫폼 정보가 없습니다.")

    print("\n구독 전략:")
    print(recommendations['subscription_strategy'].get('strategy', "구독 전략 정보가 없습니다."))

2025-05-20 21:06:46,842 - INFO - 남성 데이터 445개, 여성 데이터 445개, OTT 가격 데이터 22개 로드 완료
2025-05-20 21:06:46,852 - INFO - 12개의 플랫폼에 대한 콘텐츠 매핑 생성 완료
2025-05-20 21:06:46,853 - INFO - 사용자 프로필 생성: {'gender': '남성', 'age_group': '20대', 'preferred_genres': ['액션', '코미디'], 'detailed_genres': ['액션', '코미디', '모험'], 'weekly_hours': 10, 'monthly_budget': 15000}
2025-05-20 21:06:46,855 - WARNING - 필터링 후 데이터셋이 너무 작아 원본 데이터셋 사용
2025-05-20 21:06:46,863 - INFO - 사용자 선호도와 일치하는 콘텐츠 6개 발견
2025-05-20 21:06:46,865 - INFO - 추천 플랫폼: [{'platform': 'coupang play', 'plan': '기본', 'price': 4990}, {'platform': 'Wavve', 'plan': '베이직', 'price': 7900}], 총 비용: 12890, 월 예산: 15000


추천 콘텐츠:
- 제목: 드래곤 길들이기, 장르: 영화, 애니메이션, 플랫폼: coupang play
- 제목: 드래곤 길들이기, 장르: 영화, 애니메이션, 플랫폼: coupang play
- 제목: 드래곤 길들이기, 장르: 영화, 애니메이션, 플랫폼: coupang play
- 제목: 드래곤 길들이기, 장르: 영화, 애니메이션, 플랫폼: coupang play
- 제목: 드래곤 길들이기, 장르: 영화, 애니메이션, 플랫폼: coupang play
- 제목: 플로우, 장르: 영화, 애니메이션, 플랫폼: coupang play

추천 플랫폼:
추천 플랫폼 정보가 없습니다.

구독 전략:
하나의 플랫폼을 구독하여 콘텐츠 시청
